In [1]:
import kagglegym
from sklearn import linear_model as lm
import numpy as np
from pca import pca
import pandas as pd
import time

In [2]:
env = kagglegym.make()
observation = env.reset()
train = observation.train
mean_values = train.mean(axis=0)
train = train.fillna(mean_values)

In [3]:
def getTimeDurationSince(start):
    end = time.time()
    duration = end - start
    if duration < 60:
        return str(round(duration, 2)) + 's'
    else:
        mins = int(duration / 60)
        secs = round(duration % 60, 2)
        if mins < 60:
            return str(mins) + 'm ' + str(secs) + 's'
        else:
            hours = int(duration / 3600)
            mins = mins % 60
            return str(hours) + 'h ' + str(mins) + 'm ' + str(secs) + 's'

In [4]:
high_y_cut = 0.093497
low_y_cut = -0.086093
y_is_above_cut = (train.y > high_y_cut)
y_is_below_cut = (train.y < low_y_cut)
y_is_within_cut = (~y_is_above_cut & ~y_is_below_cut)

def run_lr(cols_to_use, reduce_components_to=-1, clip_y=False):
    starTime = time.time()
    
    env.unique_idx = int(env.n/2)
    observation = env.reset()
    train = observation.train
    mean_values = train.mean(axis=0)
    train = train.fillna(mean_values)

    model = lm.LinearRegression()
    train_x = np.array(train[cols_to_use])
    if reduce_components_to != -1:
        train_x, evecs = pca(train_x, red_cmps_to=reduce_components_to)
        
    if clip_y:
        train_x = train_x[y_is_within_cut]
        train_y = train.loc[y_is_within_cut, 'y']
    else:
        train_y = train.y.values
        
    model.fit(train_x, train_y)

    while True:
        features = observation.features.fillna(mean_values)
        test_x = np.array(features[cols_to_use])
        if reduce_components_to != -1:
            test_x_pca = np.dot(np.transpose(evecs), np.transpose(test_x))
            test_x = np.transpose(test_x_pca)
        observation.target.y = model.predict(test_x)
        if clip_y:
            observation.target.y = observation.target.y.clip(low_y_cut, high_y_cut)
        target = observation.target

        observation, reward, done, info = env.step(target)
        if done:
            break
    components = 'wszystkich' if reduce_components_to == -1 else str(reduce_components_to)
    print('Dla {} składowych głownych czas wyniósł: {}'.format(components, getTimeDurationSince(starTime)))
    return info['public_score']

In [5]:
pca_indexes = ['all', '50', '35', '20', '10', '5', '2']
def run_lr_pca(cols_to_use, clip_y=False):
    return pd.Series([
      run_lr(cols_to_use, reduce_components_to=-1, clip_y=clip_y),
      run_lr(cols_to_use, reduce_components_to=50, clip_y=clip_y),
      run_lr(cols_to_use, reduce_components_to=35, clip_y=clip_y),
      run_lr(cols_to_use, reduce_components_to=20, clip_y=clip_y),
      run_lr(cols_to_use, reduce_components_to=10, clip_y=clip_y),
      run_lr(cols_to_use, reduce_components_to=5, clip_y=clip_y),
      run_lr(cols_to_use, reduce_components_to=2, clip_y=clip_y),
    ], pca_indexes)

In [6]:
cols_to_use = [col for col in train.columns if col not in ['id','timestamp','y']]
corrcoefs = []
for col in cols_to_use:
    corrcoefs.append([col, np.corrcoef(train[col].values, train.y.values)[0, 1]])
corrcoefs = np.array(corrcoefs)

corrcoefs_vals = corrcoefs[:,1].astype(np.float)
corrcoefs_abs = np.absolute(corrcoefs_vals)
sort_indices = corrcoefs_abs.argsort()
sorted_corrcoefs = np.flip(corrcoefs[sort_indices], axis=0)
top_corrcoefs = sorted_corrcoefs[:,0]

# Obliczenia

In [7]:
all_cols_series = run_lr_pca(cols_to_use)
print(all_cols_series)
print()
print('Z obcięciem danych dla skrajnych y:')
all_cols_clip_y_series = run_lr_pca(cols_to_use, clip_y=True)
print(all_cols_clip_y_series)

Dla wszystkich składowych głownych czas wyniósł: 43.8s
Dla 50 składowych głownych czas wyniósł: 43.87s
Dla 35 składowych głownych czas wyniósł: 40.07s
Dla 20 składowych głownych czas wyniósł: 39.2s
Dla 10 składowych głownych czas wyniósł: 40.55s
Dla 5 składowych głownych czas wyniósł: 39.36s
Dla 2 składowych głownych czas wyniósł: 40.0s
all   -0.005683
50    -0.372770
35    -0.035716
20    -0.010982
10    -0.008323
5     -0.005691
2     -0.002842
dtype: float64

Z obcięciem danych dla skrajnych y:
Dla wszystkich składowych głownych czas wyniósł: 44.11s
Dla 50 składowych głownych czas wyniósł: 43.4s
Dla 35 składowych głownych czas wyniósł: 42.43s
Dla 20 składowych głownych czas wyniósł: 41.17s
Dla 10 składowych głownych czas wyniósł: 44.52s
Dla 5 składowych głownych czas wyniósł: 42.98s
Dla 2 składowych głownych czas wyniósł: 43.32s
all   -0.003614
50    -0.122021
35    -0.022460
20    -0.004849
10    -0.007705
5     -0.003614
2     -0.000923
dtype: float64


In [8]:
top_50_corr_cols = top_corrcoefs[:50]

top_50_corr_cols_series = run_lr_pca(top_50_corr_cols)
print(top_50_corr_cols_series)
print()
print('Z obcięciem danych dla skrajnych y:')
top_50_corr_cols_clip_y_series = run_lr_pca(top_50_corr_cols, clip_y=True)
print(top_50_corr_cols_clip_y_series)

Dla wszystkich składowych głownych czas wyniósł: 40.03s
Dla 50 składowych głownych czas wyniósł: 40.21s
Dla 35 składowych głownych czas wyniósł: 40.22s
Dla 20 składowych głownych czas wyniósł: 39.55s
Dla 10 składowych głownych czas wyniósł: 39.04s
Dla 5 składowych głownych czas wyniósł: 38.29s
Dla 2 składowych głownych czas wyniósł: 38.25s
all   -0.005681
50    -0.049082
35    -0.047730
20    -0.078440
10    -0.005691
5     -0.005691
2     -0.005691
dtype: float64

Z obcięciem danych dla skrajnych y:
Dla wszystkich składowych głownych czas wyniósł: 41.83s
Dla 50 składowych głownych czas wyniósł: 42.69s
Dla 35 składowych głownych czas wyniósł: 41.52s
Dla 20 składowych głownych czas wyniósł: 40.91s
Dla 10 składowych głownych czas wyniósł: 40.89s
Dla 5 składowych głownych czas wyniósł: 40.27s
Dla 2 składowych głownych czas wyniósł: 40.65s
all   -0.003608
50    -0.042973
35    -0.046961
20    -0.027870
10    -0.003610
5     -0.003610
2     -0.003610
dtype: float64


In [9]:
top_35_corr_cols = top_corrcoefs[:35]

top_35_corr_cols_series = run_lr_pca(top_35_corr_cols)
print(top_35_corr_cols_series)
print()
print('Z obcięciem danych dla skrajnych y:')
top_35_corr_cols_clip_y_series = run_lr_pca(top_35_corr_cols, clip_y=True)
print(top_35_corr_cols_clip_y_series)

Dla wszystkich składowych głownych czas wyniósł: 43.44s
Dla 50 składowych głownych czas wyniósł: 39.94s
Dla 35 składowych głownych czas wyniósł: 41.68s
Dla 20 składowych głownych czas wyniósł: 41.16s
Dla 10 składowych głownych czas wyniósł: 47.86s
Dla 5 składowych głownych czas wyniósł: 43.99s
Dla 2 składowych głownych czas wyniósł: 41.58s
all   -0.002810
50    -0.043029
35    -0.043029
20    -0.043894
10    -0.000782
5     -0.006803
2     -0.006801
dtype: float64

Z obcięciem danych dla skrajnych y:
Dla wszystkich składowych głownych czas wyniósł: 44.97s
Dla 50 składowych głownych czas wyniósł: 45.85s
Dla 35 składowych głownych czas wyniósł: 45.16s
Dla 20 składowych głownych czas wyniósł: 42.93s
Dla 10 składowych głownych czas wyniósł: 42.3s
Dla 5 składowych głownych czas wyniósł: 41.98s
Dla 2 składowych głownych czas wyniósł: 40.19s
all   -0.000891
50    -0.041580
35    -0.041580
20    -0.044007
10     0.003384
5     -0.000994
2     -0.000367
dtype: float64


In [10]:
top_20_corr_cols = top_corrcoefs[:20]

top_20_corr_cols_series = run_lr_pca(top_20_corr_cols)
print(top_20_corr_cols_series)
print()
print('Z obcięciem danych dla skrajnych y:')
top_20_corr_cols_clip_y_series = run_lr_pca(top_20_corr_cols, clip_y=True)
print(top_20_corr_cols_clip_y_series)

Dla wszystkich składowych głownych czas wyniósł: 38.37s
Dla 50 składowych głownych czas wyniósł: 39.12s
Dla 35 składowych głownych czas wyniósł: 38.74s
Dla 20 składowych głownych czas wyniósł: 39.37s
Dla 10 składowych głownych czas wyniósł: 44.22s
Dla 5 składowych głownych czas wyniósł: 53.09s
Dla 2 składowych głownych czas wyniósł: 39.61s
all   -0.144294
50    -0.144595
35    -0.144595
20    -0.144595
10    -0.033712
5      0.002815
2     -0.002974
dtype: float64

Z obcięciem danych dla skrajnych y:
Dla wszystkich składowych głownych czas wyniósł: 44.17s
Dla 50 składowych głownych czas wyniósł: 44.6s
Dla 35 składowych głownych czas wyniósł: 45.51s
Dla 20 składowych głownych czas wyniósł: 42.22s
Dla 10 składowych głownych czas wyniósł: 39.37s
Dla 5 składowych głownych czas wyniósł: 39.12s
Dla 2 składowych głownych czas wyniósł: 39.42s
all   -0.038688
50    -0.039362
35    -0.039362
20    -0.039362
10    -0.055158
5      0.004740
2     -0.000980
dtype: float64


In [11]:
top_10_corr_cols = top_corrcoefs[:10]

top_10_corr_cols_series = run_lr_pca(top_10_corr_cols)
print(top_10_corr_cols_series)
print()
print('Z obcięciem danych dla skrajnych y:')
top_10_corr_cols_clip_y_series = run_lr_pca(top_10_corr_cols, clip_y=True)
print(top_10_corr_cols_clip_y_series)

Dla wszystkich składowych głownych czas wyniósł: 37.91s
Dla 50 składowych głownych czas wyniósł: 38.54s
Dla 35 składowych głownych czas wyniósł: 39.57s
Dla 20 składowych głownych czas wyniósł: 37.54s
Dla 10 składowych głownych czas wyniósł: 39.73s
Dla 5 składowych głownych czas wyniósł: 37.95s
Dla 2 składowych głownych czas wyniósł: 37.42s
all   -0.034462
50    -0.034511
35    -0.034511
20    -0.034511
10    -0.034511
5     -0.042849
2      0.003278
dtype: float64

Z obcięciem danych dla skrajnych y:
Dla wszystkich składowych głownych czas wyniósł: 39.16s
Dla 50 składowych głownych czas wyniósł: 40.06s
Dla 35 składowych głownych czas wyniósł: 41.7s
Dla 20 składowych głownych czas wyniósł: 40.75s
Dla 10 składowych głownych czas wyniósł: 39.36s
Dla 5 składowych głownych czas wyniósł: 38.95s
Dla 2 składowych głownych czas wyniósł: 39.14s
all   -0.021434
50    -0.021495
35    -0.021495
20    -0.021495
10    -0.021495
5     -0.046948
2      0.003414
dtype: float64


In [12]:
top_5_corr_cols = top_corrcoefs[:5]

top_5_corr_cols_series = run_lr_pca(top_5_corr_cols)
print(top_5_corr_cols_series)
print()
print('Z obcięciem danych dla skrajnych y:')
top_5_corr_cols_clip_y_series = run_lr_pca(top_5_corr_cols, clip_y=True)
print(top_5_corr_cols_clip_y_series)

Dla wszystkich składowych głownych czas wyniósł: 38.01s
Dla 50 składowych głownych czas wyniósł: 37.37s
Dla 35 składowych głownych czas wyniósł: 37.67s
Dla 20 składowych głownych czas wyniósł: 38.13s
Dla 10 składowych głownych czas wyniósł: 38.66s
Dla 5 składowych głownych czas wyniósł: 38.26s
Dla 2 składowych głownych czas wyniósł: 37.63s
all   -0.042076
50    -0.042016
35    -0.042016
20    -0.042016
10    -0.042016
5     -0.042016
2      0.003316
dtype: float64

Z obcięciem danych dla skrajnych y:
Dla wszystkich składowych głownych czas wyniósł: 39.68s
Dla 50 składowych głownych czas wyniósł: 39.42s
Dla 35 składowych głownych czas wyniósł: 39.49s
Dla 20 składowych głownych czas wyniósł: 39.42s
Dla 10 składowych głownych czas wyniósł: 39.06s
Dla 5 składowych głownych czas wyniósł: 39.17s
Dla 2 składowych głownych czas wyniósł: 39.13s
all   -0.037416
50    -0.037712
35    -0.037712
20    -0.037712
10    -0.037712
5     -0.037712
2      0.003429
dtype: float64


# Podsumowanie:

In [14]:
d = {'01. Wszystkie kolumny':                                               all_cols_series,
     '02. Wszystkie kolumn z obcięciem dla skrajnych y':                    all_cols_clip_y_series,
     '03. 50 najbardziej skorelowanych kolumn':                             top_50_corr_cols_series,
     '04. 50 najbardziej skorelowanych kolumn z obcięciem dla skrajnych y': top_50_corr_cols_clip_y_series,
     '05. 35 najbardziej skorelowanych kolumn':                             top_35_corr_cols_series,
     '06. 35 najbardziej skorelowanych kolumn z obcięciem dla skrajnych y': top_35_corr_cols_clip_y_series,
     '07. 20 najbardziej skorelowanych kolumn':                             top_20_corr_cols_series,
     '08. 20 najbardziej skorelowanych kolumn z obcięciem dla skrajnych y': top_20_corr_cols_clip_y_series,
     '09. 10 najbardziej skorelowanych kolumn':                             top_10_corr_cols_series,
     '10. 10 najbardziej skorelowanych kolumn z obcięciem dla skrajnych y': top_10_corr_cols_clip_y_series,
     '11. 5 najbardziej skorelowanych kolumn':                              top_5_corr_cols_series,
     '12. 5 najbardziej skorelowanych kolumn z obcięciem dla skrajnych y':  top_5_corr_cols_clip_y_series}

df = pd.DataFrame(d, pca_indexes).transpose()
df.style.set_table_styles([{'selector': '.row_heading',
                           'props': [('text-align', 'left')]}])